# Feature

This notebook is not a feature. It explores a systematic way to create notebooks so they can reliably be exported as plain .py files that work as modules or scripts.

## Motivation
Notebooks are great. They're versatile -- you can explore and refine an idea with REPL and "rich display". Then what? 

- copy-and-paste the brilliance around, making reliable updates an impossible task
- export to python with nbconvert and risk unreliable behaviour
- export and modify so it runs reliably but reversal to continue exploring is difficult
- copy and paste specific cells into a python file which takes a bit but gives a consistent result but making reversal even more difficult

This doesn't even get into module versus running it as a script. When you run a notebook you expect it to do things. When you import a module you expect the opposite. `if...main...` works but doesn't cover everything...and is annoying to type.

Notebooks shall

- provide clear structures and functions for external use
- have no extraneous globals
- export to .py and run without ipython
- only execute `__main__` type work when 
  - in the notebook itself
  - called as a script (not `import`)
  - `%run` by another notebook
- never execute functions meant only for that specific notebook as `main` type functionality (e.g., tests)
- accept a specific clearly-defined set of configuration parameters that can be set 
  - in the notebook
  - through a function 
  - direct setting post `import`
  - via command-line arguments
- the defined configuration set is all that is required to complete the `__main__` functionality
- not be full of if/try type logic to do the above!

# Solution
Define as many parmeters up front as possible, heavy use of functional programming. Try to make as many things pure definitions and functions as possible. Then configure the few parameters that actually vary from run to run and include all 'work' in a small `if...main...` clause. 

Mark extraneous things like examples so nbconvert ignores them.

Put things like tests under an `if...main...` and probably my `if running_nb==nb_name` bit so when a notebook is `%run` by another it doesn't perform who knows what on the running notebook instead of itself.

Problem: if we mark individual cells not to run it's noisy. Metadata is hidden and annoying to change in Colab. Using cell ids is slightly less annoying, using cell number is prone to breakage.

Okaaaaay. How's this. Mostly we will want to include everything *except* cells under 'usage' or 'examples'. We can filter for that. Maybe tests, too. Just make it a toml setting or something. Then we can specifically include or exclude individual cells with #nbconvert:no and #nbconvert:yes

# Imports

In [1]:
'''Export friendly python files from notebooks'''
import os
import sys
import argparse
import inspect
import typing
from google.colab import _message
import json

In [12]:
if 'c' not in globals():
  !pip install nbf
  from nbf import epc
  c = epc.do_setup({'resources':{'nbf':{}}, 'keyring':'bit.ly/3QPP1bs'})
  %run {c.get('nb_dir')}notebooks_as_features.ipynb
  logger = epc.get_logger(c.get('nb_name'))
else:
  !cd {c['project_path']} && git pull

remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 16 (delta 9), reused 15 (delta 8), pack-reused 0
Unpacking objects: 100% (16/16), 27.28 KiB | 821.00 KiB/s, done.
From https://github.com/Jaciss/nbf
   5d9d83a..ab249ee  main       -> origin/main
Updating 5d9d83a..ab249ee
Fast-forward
 nbf/d2d.py                    | 207 +++++++++++
 nbf/gitcolab.py               | 211 ++++++++++++
 notebooks/d2d.ipynb           | 528 +++++++++++++++++++++++++++-
 notebooks/git_for_colab.ipynb | 784 +-----------------------------------------
 notebooks/ipynb2py.ipynb      |   2 +-
 5 files changed, 947 insertions(+), 785 deletions(-)
 create mode 100644 nbf/d2d.py
 create mode 100644 nbf/gitcolab.py


In [3]:
# c

# Structures

In [4]:
skip_main_on_args=['-v','-h','--version','--help']

# mkdir to save to if it doesn't exist, unfortunately needs name/name in python
c['src'] = f"{c['project_path']}{c['active_project']}/"
!mkdir -p {c['src']}

In [13]:
%%writefile {c['nb_dir']}templates/nb2py_prepend
# WARNING: this file is automatically generated, changes will be overwritten
import os
import sys
import logging
logger = logging.getLogger()
if "ipykernel_launcher" not in sys.argv[0]:
  logger.warning("get_ipython() CALLS WILL RETURN '' (empty)")
  class get_ipython():
    @classmethod
    def system(one,two=None,three=None,four=None):
      return os.system(two)
    @classmethod
    def run_cell_magic(one,two=None,three=None,four=None):
      return ''
    @classmethod
    def getoutput(one,two):
      return ''
    @classmethod
    def run_line_magic(one,two,three):
      return ''

Overwriting /content/nbf/notebooks/templates/nb2py_prepend


# Functions

In [14]:
%%writerun {c['nb_dir']}templates/nb2py_prepend -e -a
# rel: https://docs.python.org/3/library/argparse.html
# rel: https://docs.python.org/3/library/inspect.html
def argparse_from_signature(function_name, parser):
  '''uses a function signature to create argparse arguments'''
  for param,val in inspect.signature(function_name).parameters.items():
    argpd = {}
    choices = list(typing.get_args(val.annotation))
    # auto-convert float and int from cmdline
    if val.annotation==int: argpd['type']=int
    elif val.annotation==float: argpd['type']=float
    elif len(choices)>0:
      argpd['choices']=choices
    # set any defaults we know of from the signature
    if val.default!=val.empty: 
      argpd['default']=val.default
      argpd['help']='(default: %(default)s)'

    if 'variadic' in val.kind.description: argpd['nargs']='*' #variadic keyword|positional
    # if positional *and* no default then do "required" logic
    if 'positional' in val.kind.description and val.default==val.empty: parser.add_argument(val.name,**argpd)
    else: parser.add_argument('-'+val.name[0].lower(), '--'+val.name.lower(), **argpd)
  return parser

def setup_cmdline_args():
  '''automatic command line arg system using `config_dict()` 
  kwargs must be named "kwargs" to parse'''
  global c

  parser = argparse.ArgumentParser(
    description=__doc__+'\n(abbreviations e.g., --abbr: ON)',
    # width for Jupyter, avoid one word per line
    formatter_class=lambda prog: argparse.RawTextHelpFormatter(prog, width=79)
  )
  parser = argparse_from_signature(config_dict, parser)
  # caps to avoid collisions
  rdefault = 0 if '-R' in str(sys.argv) else 2
  vdefault = 0 if '-V' in str(sys.argv) else 5
  parser.add_argument('-R', '--runlevel', action='count', default=rdefault, help='repeatable, -RRR is runlevel=3 default:%(default)s')
  parser.add_argument('-V', '--verbose', action='count', default=vdefault, help='repeatable, -VVVVV is verbosity=5 default:%(default)s (1:critical,2:error,3:warning,4:info,5:debug)')
  parser.add_argument("-v", "--version", action="version",version = f"{parser.prog} version 1.0.0")
  
  args = vars(parser.parse_args())
  # keyword arg fun! split : and = delimited list items into a dict, works with JSON input
  if 'kwargs' in args and args['kwargs'] != None:
    eq2 = [i.replace(':','=') for i in args['kwargs']]
    args=dict(**args, **dict(i.split('=') for i in eq2))
    # del args['kwargs']
  c = config_dict(**args) # run args through usual config_dict: win.
  return c

overwrote a /content/nbf/notebooks/templates/nb2py_prepend ['-e', '-a']


In [ ]:
# def ():
#   json = get_json_for_python_export()
  # with open(testnb, 'w') as nb_file:
  #   nb_file.write(json.dumps(activenb,indent=2))

# Examples

In [6]:
testnb = 'testnb.ipynb'
print('first line, a rude little boy that speaks no matter what')


def config_dict(pos_req_int:int,in_both:typing.Literal['a', 'um', None]=None,only_in_sig='default',wow:typing.Union[str,int]='winz',repeat=3,**kwargs):
  if in_both == None:
    in_both = 'a'
  only_in_form=True
  final = dict(locals(),**kwargs)
  del final['kwargs']
  return final



# this logic to be added on nbconvert --to python
if "ipykernel_launcher" not in sys.argv[0]:
  c = setup_cmdline_args()
else:
  c = config_dict(3)
  print('run by ipython')
if (__name__ == '__main__') and sys.argv[1] not in skip_main_on_args:
  print('main runs here, config is:',c)
  

first line, a rude little boy that speaks no matter what
run by ipython
main runs here, config is: {'pos_req_int': 3, 'in_both': 'a', 'only_in_sig': 'default', 'wow': 'winz', 'repeat': 3, 'only_in_form': True}


write the first two cells of this nb to a ipynb file to `%%run` (no recursion ;)

## notebook runs `__main__`

Clearly it runs if you run it in the notebook, but if you `%run` it `__main__` also fires:

In [7]:
# import sys
# if (__name__ == '__main__'): # recursion
# if "ipykernel_launcher" in sys.argv[0]:
%run {testnb}

first line, a rude bit that speaks no matter what
run by ipython
main runs here, config is: {'pos_req_int': 3, 'in_both': 'a', 'only_in_sig': 'default', 'wow': 'winz', 'repeat': 3, 'only_in_form': True}


convert the previous ipynb to python to test module and script use

In [8]:
import os
testpy=testnb[:-6]+'.py' #replace ipynb with py
print(f'convert {testpy} to {testnb}')
command = f'jupyter nbconvert --to python --output-dir . --output {testpy} {testnb}'
print(command)
os.system(command)
!cat {testpy}

convert testnb.py to testnb.ipynb
jupyter nbconvert --to python --output-dir . --output testnb.py testnb.ipynb
#!/usr/bin/env python
# coding: utf-8

# # test code

# In[ ]:


'''I'm a doc description'''
import sys
import argparse
import inspect
import typing

print('first line, a rude bit that speaks no matter what')
testnb = 'testnb.ipynb'
skip_main_on_args=['-v','-h','--version','--help']


def config_dict(pos_req_int:int,in_both:typing.Literal['a', 'um', None]=None,only_in_sig='default',wow:typing.Union[str,int]='winz',repeat=3,**kwargs):
  if in_both == None:
    in_both = 'a'
  only_in_form=True
  final = dict(locals(),**kwargs)
  del final['kwargs']
  return final

def argparse_from_signature(function_name, parser):
  '''uses a function signature to create argparse arguments'''
  for param,val in inspect.signature(function_name).parameters.items():
    argpd = {}
    choices = list(typing.get_args(val.annotation))
    # auto-convert float and int from cmdline
    if val.annotati

this does not run main but *does* know it was imported by ipython

In [9]:
import importlib

if testnb in sys.modules: importlib.reload(testnb)
else: import testnb

first line, a rude bit that speaks no matter what
run by ipython


This knows it was not run by ipython

In [10]:
# !python -m pip -h


In [11]:
!python testnb.py -h

first line, a rude bit that speaks no matter what
usage: testnb.py [-h] [-i {a,um,None}] [-o ONLY_IN_SIG]
                 [-w {<class 'str'>,<class 'int'>}] [-r REPEAT]
                 [-k [KWARGS [KWARGS ...]]] [-R] [-V] [-v]
                 pos_req_int

I'm a doc description
(abbreviations e.g., --abbr: ON)

positional arguments:
  pos_req_int

optional arguments:
  -h, --help            show this help message and exit
  -i {a,um,None}, --in_both {a,um,None}
                        (default: None)
  -o ONLY_IN_SIG, --only_in_sig ONLY_IN_SIG
                        (default: default)
  -w {<class 'str'>,<class 'int'>}, --wow {<class 'str'>,<class 'int'>}
                        (default: winz)
  -r REPEAT, --repeat REPEAT
                        (default: 3)
  -k [KWARGS [KWARGS ...]], --kwargs [KWARGS [KWARGS ...]]
  -R, --runlevel        repeatable, -RRR is runlevel=3 default:2
  -V, --verbose         repeatable, -VVVVV is verbosity=5 default:5 (1:critical,2:error,3:warning,4:inf

In [12]:
!python testnb.py -v

first line, a rude bit that speaks no matter what
testnb.py version 1.0.0


In [13]:
!python testnb.py 3

first line, a rude bit that speaks no matter what
main runs here, config is: {'pos_req_int': 3, 'in_both': 'a', 'only_in_sig': 'default', 'wow': 'winz', 'repeat': 3, 'only_in_form': True, 'runlevel': 2, 'verbose': 5}


In [14]:
!python testnb.py -VVV 3 -i a -o two --kwargs a=4 b=a c=4.4 {'j':'s','o':'n',2:'cool'}

first line, a rude bit that speaks no matter what
main runs here, config is: {'pos_req_int': 3, 'in_both': 'a', 'only_in_sig': 'two', 'wow': 'winz', 'repeat': 3, 'only_in_form': True, 'runlevel': 2, 'verbose': 3, 'a': '4', 'b': 'a', 'c': '4.4', 'j': 's', 'o': 'n', '2': 'cool'}


In [15]:
!python testnb.py 3 --onl testing --rep=5 -RRR

first line, a rude bit that speaks no matter what
main runs here, config is: {'pos_req_int': 3, 'in_both': 'a', 'only_in_sig': 'testing', 'wow': 'winz', 'repeat': '5', 'only_in_form': True, 'runlevel': 3, 'verbose': 5}


In [16]:
t=['-rrr']
for grr in t:
  print(grr[0:2])
  print(grr[0:2]=='-r')



-r
True




So what do we want them to do? 
- modules: get the defs but *not* run anything
- `%run`: run things using set `c`onfig
  - but no recursion; nb_name!=running_nb ? Not in stack?
  - exceptions for tests, examples, lots of things


In [17]:
import typing

#@title Default title text { run: "auto", vertical-output: true }
def config_dict(positional_required_int_arg:int=3,conf_var:typing.Literal['a', 'um']=None,wow:typing.Union[int,str]='yay',**kwargs):
  if conf_var == None:
    conf_var = 'a' #@param ['a','um'] {type:"string"}
  final = dict(locals(),**kwargs)
  del final['kwargs']
  return final

import inspect
import typing
cdsig = inspect.signature(config_dict)
for param in cdsig.parameters:
  # print('param:',cdsig.parameters[param])
  print('name:',cdsig.parameters[param].name)
  print('default:',cdsig.parameters[param].default)
  print('isempty',cdsig.parameters[param].default==cdsig.empty)
  print('annot:',cdsig.parameters[param].annotation)
  print('annot choices:',list(typing.get_args(cdsig.parameters[param].annotation)))
  # print(str(cdsig.parameters[param].annotation))
  print('kind:',cdsig.parameters[param].kind.description)
  if cdsig.parameters[param].annotation==int: print('INT!')
  print()

# inspect.getcomments(config_dict)
# for line in inspect.getsource(config_dict).split('\n'):
#   if '@param' in line:
#     print(line)
# config_dict(1,3)
# x = typing.Union[int,str]
# isinstance('yay',typing.get_args(x))

name: positional_required_int_arg
default: 3
isempty False
annot: <class 'int'>
annot choices: []
kind: positional or keyword
INT!

name: conf_var
default: None
isempty False
annot: typing.Literal['a', 'um']
annot choices: ['a', 'um']
kind: positional or keyword

name: wow
default: yay
isempty False
annot: typing.Union[int, str]
annot choices: [<class 'int'>, <class 'str'>]
kind: positional or keyword

name: kwargs
default: <class 'inspect._empty'>
isempty True
annot: <class 'inspect._empty'>
annot choices: []
kind: variadic keyword



In [18]:
testnb.config_dict('whee',**{'one':1,'fromkwargdict':'kwarg'})
# config_dict(um=3,confvar=2)

{'pos_req_int': 'whee',
 'in_both': 'a',
 'only_in_sig': 'default',
 'wow': 'winz',
 'repeat': 3,
 'only_in_form': True,
 'one': 1,
 'fromkwargdict': 'kwarg'}

# EOF

In [8]:
if c['nb_name']==c['running_nb']:
  run_nb('git_for_colab')
  git_ppush('nb2py_prepend')

2023-01-20 12:27:48 ipynb2py INFO     /content/nbf/notebooks/ipynb2py --> /content/nbf/notebooks/git_for_colab


run_nb( git_for_colab ) nb_name: ipynb2py current running_nb set to: ipynb2py
running: git_for_colab
Already up to date.
Writing /root/.jupyter/nbdime_config.json
try to write
/content/nbf/notebooks/ipynb2py.ipynb
-rw-r--r-- 1 root root 43269 Jan 20 12:28 /content/nbf/notebooks/ipynb2py.ipynb



Already up to date.
On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   notebooks/ipynb2py.ipynb
	new file:   notebooks/templates/nb2py_prepend

message: nb2py_prepend
Do you want to commit now? y/N: y
[main 092d224] nb2py_prepend
 2 files changed, 1317 insertions(+)
 create mode 100644 notebooks/ipynb2py.ipynb
 create mode 100644 notebooks/templates/nb2py_prepend
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 11.14 KiB | 5.57 MiB/s, done.
Total 6 (delta 3), reused 0 (del